In [ ]:
import numpy as np
from astropy.io import fits
# from astropy.visualization import quantity_support
from os.path import join as jp
import matplotlib.pyplot as plt

## 1. Blackbody fitting

In [ ]:
fits_path = '../fits'

with fits.open(jp(fits_path, 'sn.mt')) as hdul:
    hdul.info()
    hdu = hdul[0]
    header = hdu.header
    data = hdu.data

Let's define a function to build the wavelength vector.

In [ ]:
def get_wl(header):
    n = header['NAXIS1']
    start = header['CRVAL1']
    delta = header['CDELT1']
    wl = np.arange(start, start + n * delta, delta)
    return wl

wl = get_wl(header)

We now define an auxiliary function to compute the blackbody spectrum. `astropy` provides the `BlackBody` class.

In [ ]:
from astropy.modeling.models import BlackBody
import astropy.units as u

def Planck(wl, T, scale=1.0):
    norm = scale * 1.0e-07 * (u.erg / (u.AA * u.s * u.sr * u.cm**2))
    # this is the normalisation of the Planck spectrum we will work with!
    bb = BlackBody(temperature=T * u.K, scale=norm)
    flux = bb(wl * u.AA) # astropy must be aware of units!
    return flux.value # but we will just work with numerical values

And finally we try to superimpose the blackbody spectrum to the measured spectrum!

In [ ]:
fig, ax = plt.subplots()
ax.plot(wl, Planck(wl, 5500, 5), label='T=5500K, scale=5')
ax.plot(wl, Planck(wl, 5500, 10), label='T=5500K, scale=10')
ax.plot(wl, data[5], label='spectrum at day 5')
# ax.set_ylabel() -> now we should know the units! (remember: fluxes are absolutely calibrated)
ax.legend()

Now you have all you need to try and 'manually' fit Planck spectra to the measured SN 1987A spectra! Be careful as both the temperature and the absolute scale (luminosity) can be adjusted!

Python has many tools for 'automatic' fitting as well (never `scipy.optimize`), you are free to use them if you feel confident enough (but I would not recommend it unless you have previous experience with minimizers).

## 2. Measurement of line shifts

We shall now try to measure the line displacement. For example, let's superimpose an hydrogen line to the measure spectrum.

In [ ]:
# Depending on your environment, you may get an interactive plotting window enabling one of the following:
# %matplotlib widget
# %matlotlib qt

day = 5

H_alpha = 6562 # AA

lines = [H_alpha,] # you can have more than one

fig, ax = plt.subplots()
ax.plot(wl, data[day-1])
for line in lines:
    ax.axvline(line, linestyle='--', color='tab:red')
# ax.set_xlim(4500, 7000) # maybe zooming around the line of interest can help!

This looks like a neat P-Cygni profile! 

From here on, you are on your own. Feel free to experiment and use the method you prefer to carry out the required measurements!